In [ ]:
!pip install PyMuPDF transformers langchain accelerate bitsandbytes einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import fitz
import torch

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
import fitz
def extract_text(pdf_path):
  pdf_document = fitz.open(pdf_path)
  text = ""
  for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    text+=page.get_text()
  pdf_document.close()
  return text

resume_text = extract_text("/content/eg_resume.pdf")

In [ ]:
resume_text

"AASHIQ H\nEDUCATION\nPROJECTS\nSKILLS\nCONTACT\nHOBBIES\nCurrent CGPA - 8 \nSRIHER University\n ( 2021 - 2025 )\n Explainable AI in Healthcare\nPreprocess clinical data \nPredict breast cancer, heart disease, and diabetes using SVM,\nLogistic Regression, Random Forest, and Decision Tree\nmodels.\nEvaluate accuracies of each model for each disease using k-\nfold cross-validation.\nApply Explainable AI (SHAP) to interpret model outputs and\nidentify key factors influencing disease predictions,\nenhancing transparency and understanding for medical\nprofessionals.\n F1 Dashboard \nIntegrated relevant F1 racing datasets, including race results, driver\ninformation, lap times, team performance, and championship standings. \nDesigned interactive visualizations to showcase race data, such as bar\ncharts, line graphs, maps, and heatmaps.\n Designed the dashboard with a visually appealing and user-friendly\ninterface, adhering to best practices for data visualization and design\nprinciples. \nD

In [ ]:
def summarize_text(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(
      inputs.input_ids,
      num_beams=4,
      min_length=120,
      max_length=250,
      length_penalty=2.0
  )
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

summary = summarize_text(resume_text)
print(summary)

Aashiq H is a 19-year-old studying third year B.Tech CSE ( AIML) at SRIHER University. He has created a calculator and todo list app in Android Studio using Kotlin. He also designed interactive visualizations to showcase race data, such as bar charts, line graphs, maps, and heatmaps. AASHIQ H is also studying Explainable AI (SHAP) to interpret model outputs and identify key factors influencing disease predictions. He hopes to make a positive impact in his chosen field of study. He lives in Chennai, India with his family.


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer=AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
from langchain import PromptTemplate, LLMChain
template = """
From the summary , assume the candidate is an expert in kotlin and python. generate the leet code coding questions related to kotlin and explainable ai to evaluvate the candidate's coding skills.
Question: {summary}
Answer: """

prompt = PromptTemplate(template=template, input_variables=['summary'])

llm_chain = LLMChain(prompt=prompt, llm=llm)

#question = " I want you to generate questions from the summary that i've provided you. along with the answers"

print(llm_chain.run(summary))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


1. What programming languages have you learned so far? (Kotlin and Python). 2. Can you provide the source codes for your calculator and todo list app? 3. How do you design interactive visualizations using Kotlin? Explain using an example of race data visualization. 4. Tell about the benefits you derive from learning Explainable AI (SHAP).
